In [1]:
import pandas as pd
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain.agents.agent_types import AgentType
import httpx
import os
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

In [2]:
API_KEY = ""

In [3]:
llm = ChatOpenAI(openai_api_key=API_KEY, temperature=0, model="gpt-4o-2024-08-06", max_tokens=500, http_client = httpx.Client(verify=False))

In [5]:
# Load the CSV file into a Pandas DataFrame
csv_file_path = "../data/Hospital_Information_with_Lab_Tests.csv"
df = pd.read_csv(csv_file_path)
df.head()

,index,Provider ID,Hospital Name,Address,City,State,ZIP Code,County Name,Phone Number,Hospital Type,...,Effectiveness of care national comparison,Effectiveness of care national comparison footnote,Timeliness of care national comparison,Timeliness of care national comparison footnote,Efficient use of medical imaging national comparison,Efficient use of medical imaging national comparison footnote,Location,Diagnostic Test,Health Package,Preparation Instructions
0,0,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,2565938310,Acute Care Hospitals,...,Same as the National average,NaN,Above the National average,NaN,Below the National average,NaN,"2505 U S HIGHWAY 431 NORTH\nBOAZ, AL 35957\n",MRI Scan,Cancer Screening,Wear loose and comfortable clothing.
1,1,10032,WEDOWEE HOSPITAL,209 NORTH MAIN STREET,WEDOWEE,AL,36278,RANDOLPH,2563572111,Acute Care Hospitals,...,Same as the National average,NaN,Same as the National average,NaN,Not Available,Results are not available for this reporting p...,"209 NORTH MAIN STREET\nWEDOWEE, AL 36278\n",MRI Scan,Women's Wellness Package,Arrive 30 minutes early for paperwork.
2,2,10131,CRESTWOOD MEDICAL CENTER,ONE HOSPITAL DR SE,HUNTSVILLE,AL,35801,MADISON,2568823100,Acute Care Hospitals,...,Same as the National average,NaN,Same as the National average,NaN,Same as the National average,NaN,"ONE HOSPITAL DR SE\nHUNTSVILLE, AL 35801\n",ECG,Diabetes Management,Avoid caffeine for 24 hours.
3,3,20001,PROVIDENCE ALASKA MEDICAL CENTER,BOX 196604,ANCHORAGE,AK,99508,ANCHORAGE,9072123675,Acute Care Hospitals,...,Below the National average,NaN,Below the National average,NaN,Same as the National average,NaN,"BOX 196604\nANCHORAGE, AK 99508\n",MRI Scan,Full Body Checkup,Do not eat or drink anything except water for ...
4,4,20018,YUKON KUSKOKWIM DELTA REG HOSPITAL,PO BOX 287,BETHEL,AK,99559,BETHEL,9075436300,Acute Care Hospitals,...,Below the National average,NaN,Not Available,Results are not available for this reporting p...,Not Available,Results are not available for this reporting p...,"PO BOX 287\nBETHEL, AK 99559\n",X-Ray,Full Body Checkup,Avoid caffeine for 24 hours.


In [6]:
system_message = SystemMessagePromptTemplate.from_template(
"""
You are a highly skilled healthcare assistant with expertise in suggesting health screening tests and packagaes. 
Your task is to assess various hospitals based on a user's specific conditions, preferences, and needs. 
You will evaluate hospitals considering factors such as medical specialties, patient reviews, location, cost, accessibility, facilities, 
and the availability of treatment for specific conditions.

When comparing hospitals, follow these guidelines:

- Condition-Specific Comparison: Focus on the hospitals' expertise in treating the user's specific health condition 
(e.g., heart disease, cancer, etc.).
- Hospital Features: Include details about the hospital's reputation, technology, facilities, specialized care, and any awards or 
recognitions.
- Location and Accessibility: Consider the proximity to the user’s location and the convenience of travel.
- Cost and Insurance: Compare the cost of treatment and insurance coverage options offered by the hospitals.
- Patient Feedback: Analyze reviews and ratings to gauge patient satisfaction and outcomes.
- Personalized Recommendation: Provide a clear, personalized suggestion based on the user’s priorities, whether they are medical 
expertise, convenience, or cost.

CAREFULLY look at each column name to understand what to output.
"""
)

prompt = ChatPromptTemplate.from_messages([system_message])

In [7]:
agent = create_pandas_dataframe_agent(llm, df, prompt=prompt, verbose=False, allow_dangerous_code=True, 
                                      agent_type=AgentType.OPENAI_FUNCTIONS,)

/opt/anaconda3/lib/python3.12/site-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'prompt': ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="\nYou are a highly skilled healthcare assistant with expertise in suggesting health screening tests and packagaes. \nYour task is to assess various hospitals based on a user's specific conditions, preferences, and needs. \nYou will evaluate hospitals considering factors such as medical specialties, patient reviews, location, cost, accessibility, facilities, \nand the availability of treatment for specific conditions.\n\nWhen comparing hospitals, follow these guidelines:\n\n- Condition-Specific Comparison: Focus on the hospitals' expertise in treating the user's specific health condition \n(e.g., heart disease, cancer, etc.).\n-

In [8]:
print(agent.invoke("what would be preparation instructions for cancer screening")['output'])

The preparation instructions for cancer screening are to wear loose and comfortable clothing.


In [9]:
print(agent.invoke("please suggest a good full body checkup nearby san mateo")['output'])

Here are some hospitals in California offering a "Full Body Checkup" that are relatively close to San Mateo:

1. **California Pacific Medical Center - St. Luke's Campus**
   - Address: 3555 Cesar Chavez Street, San Francisco, CA 94110
   - Phone Number: 415-641-6562

2. **San Ramon Regional Medical Center**
   - Address: 6001 Norris Canyon Road, San Ramon, CA 94583
   - Phone Number: 925-275-9200

3. **O'Connor Hospital**
   - Address: 2105 Forest Avenue, San Jose, CA 95128
   - Phone Number: 408-947-2500

4. **Kaiser Foundation Hospital - Walnut Creek**
   - Address: 1425 S Main Street, Walnut Creek, CA 94596
   - Phone Number: 925-295-4000

5. **Marin General Hospital**
   - Address: 250 Bon Air Road, PO Box 8010, Greenbrae, CA 94904
   - Phone Number: 415-925-7900

These hospitals are within a reasonable distance from San Mateo and offer full body checkup services.
